In [5]:

# coding: utf-8

from __future__ import print_function
import os
import numpy as np
from numpy import newaxis

np.random.seed(1337)

import keras

from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Convolution1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model,Sequential
from keras.optimizers import SGD
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report,recall_score,accuracy_score,confusion_matrix,roc_curve,roc_auc_score
import sys

import pandas as pd
import random


#Read MFCC data
print("Data mfcc read started...")
data = pd.read_csv("ComParE2017_Cold.ComParE.train__2.arff",delimiter=',')
data=data.as_matrix()
print ("Data mfcc read finished.")


data=data[:,1:6375]


# In[44]:

Y_train=[x[6373] for x in data]
Y_train[300]

x_train=data[:,0:6373]

labels = ['C','NC']
label2ind = {label: (index + 1) for index, label in enumerate(labels)}
ind2label = {(index + 1): label for index, label in enumerate(labels)}

print (label2ind)


# In[63]:

max_label = max(label2ind.values())+1
y_enc = [[label2ind[ey] for ey in Y_train]]


# In[65]:

def encode(x, n):
    result = np.zeros(n)
    result[x] = 1
    return result

y_enc = [[encode(c, max_label) for c in ey] for ey in y_enc]


# In[71]:

y_enc[0]
y_enci =[]
for i in (range(len(y_enc))):
    a=y_enc[i]
    v=np.array(a)
    v=np.delete(v, 0, 1)
    y_enci.append(v)

y_train=y_enci[0]




#Read MFCC data
print("Data mfcc read started...")
data2 = pd.read_csv("ComParE2017_Cold.ComParE.devel___2.arff",delimiter=',')
data2=data2.as_matrix()
print ("Data mfcc read finished.")


data2=data2[:,1:6375]


# In[44]:

Y_val=[x[6373] for x in data2]
x_val=data2[:,0:6373]

max_label = max(label2ind.values())+1
y_enc = [[label2ind[ey] for ey in Y_val]]
y_enc = [[encode(c, max_label) for c in ey] for ey in y_enc]

y_enci =[]
for i in (range(len(y_enc))):
    a=y_enc[i]
    v=np.array(a)
    v=np.delete(v, 0, 1)
    y_enci.append(v)

y_val=y_enci[0]

from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(x_train)


train_data_input = scaler.transform(x_train)
valid_data_input = scaler.transform(x_val)
test_data_input = scaler.transform(x_val)

#train_data_input = x_train
#valid_data_input = x_val
#test_data_input = x_val


train_data_target = y_train
valid_data_target = y_val
test_data_target = y_val
# In[80]:
print(train_data_input.shape)
print(test_data_input.shape)
print(valid_data_input.shape)

print(train_data_target.shape)
print(test_data_target.shape)
print(valid_data_target.shape)

#train_data_input = train_data_input[:,0:6373:2]
#valid_data_input = valid_data_input[:,0:6373:2]
#test_data_input = test_data_input[:,0:6373:2]

#print(train_data_input.shape)
#print(test_data_input.shape)
#print(valid_data_input.shape)

#print(train_data_target.shape)
#print(test_data_target.shape)
#print(valid_data_target.shape)



# on non-standardized data
pca = PCA(n_components=20).fit(train_data_input)
train_data_input = pca.transform(train_data_input)
valid_data_input = pca.transform(valid_data_input)
test_data_input = pca.transform(test_data_input)


# In[236]:

def create_class_weight(labels_dict):
    total = sum(labels_dict.values())
    keys = labels_dict.keys()
    class_weight = dict()

    for key in keys:
        score = total/float(labels_dict[key])
        class_weight[key] = score

    return class_weight

label_count={}
for i in range(train_data_target.shape[-1]):
    label_count.update({int(i):len(train_data_target[train_data_target[:,int(i)]==1])})

cweights=create_class_weight(label_count)


earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')


# In[87]:

model = Sequential()
from keras.layers import Convolution1D, MaxPooling1D, Embedding, Dropout

model.add(Dense(500, input_shape=(20,), init='glorot_normal'))
model.add(Dropout(0.5))
model.add(Dense(500, activation='relu', init='glorot_normal'))
model.add(Dropout(0.5))
model.add(Dense(500, activation='relu', init='glorot_normal'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


# In[88]:

batch_size=100
epochs = 20

#sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['acc'])
model.fit(train_data_input, train_data_target, nb_epoch=epochs,batch_size=batch_size, callbacks=[earlyStopping], shuffle=True, validation_data = (valid_data_input, valid_data_target))#class_weight=cweights)



score = model.evaluate(test_data_input, test_data_target, batch_size=batch_size) #[loss,accuracy]
accuracy = score[1]
loss = score[0]
print("Accuracy: ", accuracy, "  Loss: ", loss)

pr = model.predict_classes(test_data_input)
yh = test_data_target.argmax(1)
print("\n")
print (recall_score(pr, yh, average="macro"))


Data mfcc read started...
Data mfcc read finished.
{'C': 1, 'NC': 2}
Data mfcc read started...
Data mfcc read finished.
(9504, 6373)
(9595, 6373)
(9595, 6373)
(9504, 2)
(9595, 2)
(9595, 2)
Train on 9504 samples, validate on 9595 samples
Epoch 1/20
9504/9504 [==============================] - 2s - loss: 1.6536 - acc: 0.8943 - val_loss: 1.6891 - val_acc: 0.8946
Epoch 2/20
9504/9504 [==============================] - 1s - loss: 1.6340 - acc: 0.8980 - val_loss: 1.6890 - val_acc: 0.8946
Epoch 3/20
9504/9504 [==============================] - 1s - loss: 1.5868 - acc: 0.8922 - val_loss: 1.6326 - val_acc: 0.8598
Epoch 4/20
9504/9504 [==============================] - 1s - loss: 1.1633 - acc: 0.8709 - val_loss: 0.6999 - val_acc: 0.8946
Epoch 5/20
9504/9504 [==============================] - 1s - loss: 0.5167 - acc: 0.8697 - val_loss: 0.3493 - val_acc: 0.8967
Epoch 6/20
9504/9504 [==============================] - 1s - loss: 0.3559 - acc: 0.8923 - val_loss: 0.4201 - val_acc: 0.8727
Epoch 7/20
95